In [1]:
# 先从简单的CNN开始写吧，嘶，主要是数据太少了orzzzzz
# 所以只能做分类了，完蛋，我开始写之前怎么就没想到这点呢
# 没事，3个类，33个数据一类，勉勉强强够了，大不了我自己再洗一年的数据

import pandas as pd
from utils import randomSplit, Accumulator
from torch import nn
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler  # 归一化


def rank2class(df):
    if df['LgRk'] <= 6:
        return 0
    elif 6 < df['LgRk'] <= 13:
        return 1
    else:
        return 2


In [2]:
class myDataset():
    def __init__(self):
        self.player = pd.read_csv('datacleaning/modified_data.csv', header=0, encoding='unicode_escape', delimiter=';')
        self.player = self.player.drop(['Rk', 'Player', 'Nation', 'Pos', 'Comp', 'Age', 'Born'], axis=1)
        scaler = StandardScaler()  # z score 归一化
        for col in self.player.columns:
            if col == 'Squad': continue
            self.player[[col]] = scaler.fit_transform(self.player[[col]])


        self.league = pd.read_excel('datacleaning/Big 5 European Leagues.xlsx', sheet_name='Big 5 European Leagues Stats')
        self.league['tier'] = self.league.apply(rank2class, axis=1)
        self.league = self.league[['Squad', 'tier']].values             
        # 划分集合
        self.trainSet, self.validSet, self.testSet = randomSplit(self.league, part=(0.65, 1.0, 1.0))
        self.dataSource = dict()
        self.dataSource['train'] = self.trainSet
        self.dataSource['valid'] = self.validSet
        self.dataSource['test'] = self.testSet


    def Squad2Player(self, squad):
        "输入squad的名字，输出球队球员的数据"
        re = self.player[self.player['Squad'] == squad].drop('Squad', axis=1)
        re = torch.Tensor(re.values)

        # 下面是reshape，在这里就做了吧
        # return shape [1, 40, 135]
        if re.shape[0] < 40:
            a = (40 - re.shape[0]) // 2
            b = 40 - re.shape[0] - a
            a = torch.zeros((a, re.shape[1]))
            b = torch.zeros((b, re.shape[1]))
            return torch.cat([a, re, b], dim=0).unsqueeze(0)
        elif re.shape[0] == 40:
            return re.unsqueeze(0)
        else:
            return re[:40, :].unsqueeze(0)
        

    
    def DataIter(self, batchsize, source, shuffle=True):
        assert source in ('train', 'valid', 'test')
        data = self.dataSource[source]
        if shuffle:
            np.random.shuffle(data)
        for i in range(0, data.shape[0], batchsize):
            upper = min(i + batchsize, data.shape[0] - 1)
            x_l = []
            y_l = []
            for j in range(i, upper):
                x_l.append(self.Squad2Player(data[j, 0]).unsqueeze(0))
                y_l.append(data[j, 1])
            yield torch.cat(x_l, dim=0), torch.LongTensor(y_l)



In [3]:
def train(net, dataset, batch_size, num_epochs, lr, device):
    # 初始化参数
    def init_weight(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weight)
    
    print(f'training on: {device}')
    net.to(device)

    optimizer = torch.optim.Adam([p for p in net.parameters() if p.requires_grad==True], lr=lr)
    loss = nn.CrossEntropyLoss(label_smoothing=0.1)

    res = [] # 训练集梯度，训练集正确率，训练集总数
    for epoch in range(num_epochs):
        recorder = Accumulator(3) # 训练集梯度，训练集正确率，训练集总数
        net.train()
        for X, y in dataset.DataIter(batch_size, 'train'):
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            yhat = net(X)
            l = loss(yhat, y)
            l.backward()
            optimizer.step()
            
            correct = (yhat.argmax(axis=1) == y).sum()
            acc = int(correct) / int(y.shape[0])

            recorder.add(float(l * X.shape[0]), correct, X.shape[0]) #说明lossfun里，梯度除以了bachsize

        # 正确率和平均损失
        net.eval()
        validRecorder = Accumulator(3)
        for X, y in dataset.DataIter(batch_size, 'valid'):
            X, y = X.to(device), y.to(device)
            yhat = net(X)
            l = loss(yhat, y)
            correct = (yhat.argmax(axis=1) == y).sum()
            acc = int(correct) / int(y.shape[0])
            validRecorder.add(float(l * X.shape[0]), correct, X.shape[0]) #说明lossfun里，梯度除以了bachsize


        print(f'epoch = {epoch+1}')
        print(f'训练集正确率：{recorder[1]/recorder[2]:.3f}，训练集平均loss {recorder[0]/recorder[2]:.3f}.', end=' ')
        print(f'验证集正确率：{validRecorder[1]/validRecorder[2]:.3f}，验证集平均loss {validRecorder[0]/validRecorder[2]:.3f}.')
        
    return res



In [4]:
# 定义网络
CNNclassifier = torch.nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=1, kernel_size=7, padding=(0, 1), stride=3), # [1, 40, 135] -> [1, 12, 44]
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),   # [1, 12, 44] -> [1, 6, 22]
    nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=2),   # [1, 6, 22] -> [1, 2, 10]
    nn.Flatten(),
    nn.Linear(20, 12)
    #, nn.ReLU(), nn.Dropout(p=0.5),
    #nn.Linear(12, 3)
    )

x = torch.rand((40, 1, 40, 135))
print(CNNclassifier(x).shape)

torch.Size([40, 12])


In [5]:
dataset = myDataset()
# 训练超参数
lr = 0.05
num_epoch = 10
batch_size = 5
# 正式训练
data_l = train(CNNclassifier, dataset, batch_size, num_epoch, lr, device='cpu')

training on: cpu
epoch = 1
训练集正确率：0.339，训练集平均loss 1.902. 验证集正确率：0.294，验证集平均loss 2.082.
epoch = 2
训练集正确率：0.613，训练集平均loss 1.416. 验证集正确率：0.294，验证集平均loss 1.704.
epoch = 3
训练集正确率：0.726，训练集平均loss 1.226. 验证集正确率：0.441，验证集平均loss 1.545.
epoch = 4
训练集正确率：0.726，训练集平均loss 1.132. 验证集正确率：0.559，验证集平均loss 1.524.
epoch = 5
训练集正确率：0.855，训练集平均loss 0.993. 验证集正确率：0.559，验证集平均loss 1.560.
epoch = 6
训练集正确率：0.806，训练集平均loss 0.946. 验证集正确率：0.529，验证集平均loss 1.688.
epoch = 7
训练集正确率：0.903，训练集平均loss 0.849. 验证集正确率：0.500，验证集平均loss 1.841.
epoch = 8
训练集正确率：0.935，训练集平均loss 0.757. 验证集正确率：0.618，验证集平均loss 1.883.
epoch = 9
训练集正确率：0.935，训练集平均loss 0.737. 验证集正确率：0.559，验证集平均loss 2.081.
epoch = 10
训练集正确率：0.968，训练集平均loss 0.722. 验证集正确率：0.529，验证集平均loss 2.158.
